In [13]:
import pandas as pd
import tqdm
import warnings 
import glob
import numpy as np
from tqdm import tqdm
import seaborn as sns
import os
warnings.filterwarnings("ignore")

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import SGD

Using TensorFlow backend.
W1009 15:28:56.452131 20308 __init__.py:690] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


## X Train

In [15]:
all_train_cases = os.listdir('D:\\DeepMind\\合肥高新杯心电人机智能大赛\\train\\')
all_train_cases.sort(key=lambda x:int(x[:-4]))
all_train_cases

['2.txt',
 '5.txt',
 '9.txt',
 '12.txt',
 '14.txt',
 '16.txt',
 '17.txt',
 '21.txt',
 '23.txt',
 '24.txt',
 '26.txt',
 '28.txt',
 '30.txt',
 '32.txt',
 '34.txt',
 '36.txt',
 '40.txt',
 '41.txt',
 '49.txt',
 '53.txt',
 '55.txt',
 '57.txt',
 '60.txt',
 '61.txt',
 '63.txt',
 '65.txt',
 '68.txt',
 '71.txt',
 '72.txt',
 '76.txt',
 '78.txt',
 '81.txt',
 '84.txt',
 '86.txt',
 '89.txt',
 '90.txt',
 '92.txt',
 '95.txt',
 '97.txt',
 '98.txt',
 '100.txt',
 '102.txt',
 '105.txt',
 '108.txt',
 '110.txt',
 '112.txt',
 '114.txt',
 '116.txt',
 '118.txt',
 '121.txt',
 '123.txt',
 '124.txt',
 '126.txt',
 '128.txt',
 '130.txt',
 '132.txt',
 '135.txt',
 '137.txt',
 '141.txt',
 '144.txt',
 '145.txt',
 '147.txt',
 '149.txt',
 '151.txt',
 '154.txt',
 '156.txt',
 '158.txt',
 '160.txt',
 '161.txt',
 '163.txt',
 '165.txt',
 '167.txt',
 '169.txt',
 '171.txt',
 '173.txt',
 '175.txt',
 '177.txt',
 '179.txt',
 '182.txt',
 '184.txt',
 '188.txt',
 '189.txt',
 '191.txt',
 '194.txt',
 '197.txt',
 '199.txt',
 '203.txt',

In [16]:
x_train = np.empty(shape=[24106, 5000, 12], dtype=np.int64)

In [17]:
for idx, sample in enumerate(tqdm(all_train_cases)):
    data = pd.read_table('train\\'+sample,header=0, encoding='utf-8', sep=' ')
    i = data['I']
    ii = data['II']
    data['III'] = ii-i
    data['aVR']=-(ii+i)/2
    data['aVL']=(i-ii)/2
    data['aVF']=(ii-i)/2
    x_train[idx] = data.values

100%|███████████████████████████████████████████████████████████████████████████| 24106/24106 [03:34<00:00, 112.38it/s]


In [18]:
x_train[0]

array([[-5,  1,  9, ...,  2, -3,  3],
       [-4,  1,  9, ...,  1, -2,  2],
       [-3,  1,  9, ...,  1, -2,  2],
       ...,
       [ 4,  2, 27, ..., -3,  1, -1],
       [ 6,  3, 28, ..., -4,  1, -1],
       [ 7,  4, 29, ..., -5,  1, -1]], dtype=int64)

## Y Train

In [19]:
label = pd.read_csv('hf_round1_label.txt',sep="\t",encoding='utf-8',names = list(range(0,12)))
label.columns = ['id','age','gender','f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10','f11']
arrythmia = pd.read_csv('hf_round1_arrythmia.txt',header=None,sep="\t",encoding='utf-8')
arrythmia.columns = ['ff1']
all_dis = list(arrythmia.ff1.unique())

In [20]:
dis_df = pd.DataFrame(np.zeros([label.shape[0],55]))
dis_df.columns = all_dis
new_data = pd.concat([label,dis_df],axis =1)
for i in tqdm(range (new_data.shape[0])):
    dises = list(label.iloc[i,3:11].unique())
    dises.remove(dises[-1])
    for d in dises:
        new_data.loc[i,d]=1

100%|███████████████████████████████████████████████████████████████████████████| 24106/24106 [00:30<00:00, 785.24it/s]


In [21]:
new_cols = [col for col in new_data.columns if col not in ['id','age','gender','f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10','f11']]#最后过滤掉不需要的列new_data= new_data[new_cols]
y_train = new_data[new_cols]
y_train = y_train.values

In [22]:
y_train.shape

(24106, 55)

## X Test

In [23]:
all_test_cases = os.listdir('D:\\DeepMind\\合肥高新杯心电人机智能大赛\\testA\\')
all_test_cases.sort(key=lambda x:int(x[:-4]))
all_test_cases

['1.txt',
 '3.txt',
 '6.txt',
 '7.txt',
 '10.txt',
 '13.txt',
 '15.txt',
 '20.txt',
 '22.txt',
 '35.txt',
 '54.txt',
 '58.txt',
 '69.txt',
 '74.txt',
 '75.txt',
 '77.txt',
 '79.txt',
 '83.txt',
 '115.txt',
 '125.txt',
 '127.txt',
 '129.txt',
 '131.txt',
 '134.txt',
 '136.txt',
 '140.txt',
 '142.txt',
 '146.txt',
 '148.txt',
 '150.txt',
 '155.txt',
 '162.txt',
 '164.txt',
 '166.txt',
 '168.txt',
 '172.txt',
 '174.txt',
 '178.txt',
 '181.txt',
 '187.txt',
 '190.txt',
 '193.txt',
 '198.txt',
 '215.txt',
 '216.txt',
 '221.txt',
 '232.txt',
 '235.txt',
 '244.txt',
 '251.txt',
 '255.txt',
 '259.txt',
 '277.txt',
 '285.txt',
 '287.txt',
 '294.txt',
 '296.txt',
 '298.txt',
 '301.txt',
 '304.txt',
 '306.txt',
 '308.txt',
 '312.txt',
 '319.txt',
 '321.txt',
 '325.txt',
 '330.txt',
 '336.txt',
 '343.txt',
 '348.txt',
 '350.txt',
 '353.txt',
 '355.txt',
 '357.txt',
 '359.txt',
 '366.txt',
 '370.txt',
 '390.txt',
 '392.txt',
 '396.txt',
 '399.txt',
 '408.txt',
 '412.txt',
 '414.txt',
 '422.txt',
 '

In [24]:
x_test = np.empty(shape=[8036, 5000, 12], dtype=np.int64)

In [25]:
for idx, sample in enumerate(tqdm(all_test_cases)):
    data = pd.read_table('testA\\'+sample,header=0, encoding='utf-8', sep=' ')
    i = data['I']
    ii = data['II']
    data['III'] = ii-i
    data['aVR']=-(ii+i)/2
    data['aVL']=(i-ii)/2
    data['aVF']=(ii-i)/2
    x_test[idx] = data.values

100%|█████████████████████████████████████████████████████████████████████████████| 8036/8036 [01:15<00:00, 106.75it/s]


In [26]:
x_test[0]

array([[  4,  17,  -6, ..., -10,  -6,   6],
       [  3,  15,  -6, ...,  -9,  -6,   6],
       [  3,  17,  -7, ..., -10,  -7,   7],
       ...,
       [ -5, -11,   0, ...,   8,   3,  -3],
       [ -5, -11,   0, ...,   8,   3,  -3],
       [ -5, -11,   0, ...,   8,   3,  -3]], dtype=int64)

## X&Y Dev

In [27]:
per = np.random.permutation(y_train.shape[0])
x_train = x_train[per,:,:]
y_train = y_train[per]

In [28]:
x_train_ = x_train[:20000]
y_train_ = y_train[:20000]
x_dev_ = x_train[20000:]
y_dev_ = y_train[20000:]

## LSTM Model

In [30]:
model = Sequential()
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(55, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Adam')

model.fit(x_train_, y_train_, batch_size=32, epochs=3)

Epoch 1/3


10016/20000 [==============>...............] - ETA: 29:57 - loss: 0.72 - ETA: 27:05 - loss: 0.73 - ETA: 26:04 - loss: 0.72 - ETA: 28:25 - loss: 0.72 - ETA: 31:02 - loss: 0.72 - ETA: 33:01 - loss: 0.72 - ETA: 34:29 - loss: 0.72 - ETA: 35:35 - loss: 0.71 - ETA: 36:35 - loss: 0.71 - ETA: 37:25 - loss: 0.71 - ETA: 38:29 - loss: 0.71 - ETA: 38:54 - loss: 0.71 - ETA: 39:28 - loss: 0.71 - ETA: 39:52 - loss: 0.71 - ETA: 40:09 - loss: 0.70 - ETA: 40:29 - loss: 0.70 - ETA: 40:43 - loss: 0.70 - ETA: 40:54 - loss: 0.70 - ETA: 41:06 - loss: 0.70 - ETA: 41:18 - loss: 0.70 - ETA: 41:33 - loss: 0.69 - ETA: 41:44 - loss: 0.69 - ETA: 41:54 - loss: 0.69 - ETA: 42:00 - loss: 0.69 - ETA: 42:09 - loss: 0.69 - ETA: 42:17 - loss: 0.69 - ETA: 42:21 - loss: 0.68 - ETA: 42:30 - loss: 0.68 - ETA: 42:41 - loss: 0.68 - ETA: 42:47 - loss: 0.68 - ETA: 42:48 - loss: 0.68 - ETA: 42:48 - loss: 0.68 - ETA: 42:48 - loss: 0.68 - ETA: 42:47 - loss: 0.67 - ETA: 42:47 - loss: 0.67 - ETA: 42:46 - loss: 0.67 - ETA: 42:44 - loss

20000/20000 [==============================] - ETA: 23:18 - loss: 0.33 - ETA: 23:13 - loss: 0.33 - ETA: 23:08 - loss: 0.33 - ETA: 23:04 - loss: 0.33 - ETA: 23:00 - loss: 0.33 - ETA: 22:55 - loss: 0.33 - ETA: 22:51 - loss: 0.33 - ETA: 22:47 - loss: 0.33 - ETA: 22:42 - loss: 0.32 - ETA: 22:38 - loss: 0.32 - ETA: 22:33 - loss: 0.32 - ETA: 22:29 - loss: 0.32 - ETA: 22:24 - loss: 0.32 - ETA: 22:21 - loss: 0.32 - ETA: 22:16 - loss: 0.32 - ETA: 22:12 - loss: 0.32 - ETA: 22:07 - loss: 0.32 - ETA: 22:03 - loss: 0.32 - ETA: 21:58 - loss: 0.32 - ETA: 21:54 - loss: 0.32 - ETA: 21:49 - loss: 0.32 - ETA: 21:45 - loss: 0.32 - ETA: 21:41 - loss: 0.32 - ETA: 21:36 - loss: 0.32 - ETA: 21:32 - loss: 0.32 - ETA: 21:27 - loss: 0.32 - ETA: 21:23 - loss: 0.31 - ETA: 21:18 - loss: 0.31 - ETA: 21:13 - loss: 0.31 - ETA: 21:09 - loss: 0.31 - ETA: 21:05 - loss: 0.31 - ETA: 21:00 - loss: 0.31 - ETA: 20:56 - loss: 0.31 - ETA: 20:51 - loss: 0.31 - ETA: 20:47 - loss: 0.31 - ETA: 20:43 - loss: 0.31 - ETA: 20:38 - loss

10048/20000 [==============>...............] - ETA: 57:22 - loss: 0.12 - ETA: 54:45 - loss: 0.13 - ETA: 53:59 - loss: 0.13 - ETA: 52:16 - loss: 0.12 - ETA: 51:53 - loss: 0.12 - ETA: 51:54 - loss: 0.12 - ETA: 51:27 - loss: 0.12 - ETA: 51:25 - loss: 0.12 - ETA: 51:32 - loss: 0.12 - ETA: 51:45 - loss: 0.12 - ETA: 51:54 - loss: 0.12 - ETA: 51:35 - loss: 0.12 - ETA: 51:05 - loss: 0.12 - ETA: 50:33 - loss: 0.12 - ETA: 50:08 - loss: 0.12 - ETA: 49:41 - loss: 0.12 - ETA: 49:20 - loss: 0.12 - ETA: 48:59 - loss: 0.12 - ETA: 48:40 - loss: 0.12 - ETA: 48:24 - loss: 0.12 - ETA: 48:08 - loss: 0.12 - ETA: 47:55 - loss: 0.12 - ETA: 47:41 - loss: 0.12 - ETA: 47:28 - loss: 0.12 - ETA: 47:17 - loss: 0.12 - ETA: 47:09 - loss: 0.12 - ETA: 46:59 - loss: 0.12 - ETA: 46:50 - loss: 0.12 - ETA: 46:41 - loss: 0.12 - ETA: 46:33 - loss: 0.12 - ETA: 46:22 - loss: 0.12 - ETA: 46:14 - loss: 0.12 - ETA: 46:05 - loss: 0.12 - ETA: 45:58 - loss: 0.12 - ETA: 45:51 - loss: 0.12 - ETA: 45:51 - loss: 0.12 - ETA: 45:49 - loss

20000/20000 [==============================] - ETA: 23:18 - loss: 0.12 - ETA: 23:14 - loss: 0.12 - ETA: 23:09 - loss: 0.12 - ETA: 23:04 - loss: 0.12 - ETA: 23:00 - loss: 0.12 - ETA: 22:55 - loss: 0.12 - ETA: 22:51 - loss: 0.12 - ETA: 22:46 - loss: 0.12 - ETA: 22:42 - loss: 0.12 - ETA: 22:38 - loss: 0.12 - ETA: 22:33 - loss: 0.12 - ETA: 22:29 - loss: 0.12 - ETA: 22:24 - loss: 0.12 - ETA: 22:20 - loss: 0.12 - ETA: 22:15 - loss: 0.12 - ETA: 22:11 - loss: 0.12 - ETA: 22:06 - loss: 0.12 - ETA: 22:01 - loss: 0.12 - ETA: 21:57 - loss: 0.12 - ETA: 21:52 - loss: 0.12 - ETA: 21:48 - loss: 0.12 - ETA: 21:43 - loss: 0.12 - ETA: 21:39 - loss: 0.12 - ETA: 21:35 - loss: 0.12 - ETA: 21:30 - loss: 0.12 - ETA: 21:26 - loss: 0.12 - ETA: 21:22 - loss: 0.12 - ETA: 21:17 - loss: 0.12 - ETA: 21:13 - loss: 0.12 - ETA: 21:08 - loss: 0.12 - ETA: 21:04 - loss: 0.12 - ETA: 20:59 - loss: 0.12 - ETA: 20:55 - loss: 0.12 - ETA: 20:50 - loss: 0.12 - ETA: 20:45 - loss: 0.12 - ETA: 20:41 - loss: 0.12 - ETA: 20:36 - loss

10048/20000 [==============>...............] - ETA: 46:50 - loss: 0.10 - ETA: 47:00 - loss: 0.11 - ETA: 46:46 - loss: 0.11 - ETA: 46:33 - loss: 0.12 - ETA: 46:23 - loss: 0.12 - ETA: 46:22 - loss: 0.12 - ETA: 46:23 - loss: 0.11 - ETA: 46:22 - loss: 0.11 - ETA: 46:15 - loss: 0.11 - ETA: 46:09 - loss: 0.11 - ETA: 46:03 - loss: 0.11 - ETA: 45:59 - loss: 0.11 - ETA: 45:54 - loss: 0.11 - ETA: 45:48 - loss: 0.11 - ETA: 45:44 - loss: 0.11 - ETA: 45:39 - loss: 0.11 - ETA: 45:37 - loss: 0.11 - ETA: 45:31 - loss: 0.11 - ETA: 45:24 - loss: 0.11 - ETA: 45:20 - loss: 0.11 - ETA: 45:14 - loss: 0.11 - ETA: 45:08 - loss: 0.11 - ETA: 45:04 - loss: 0.11 - ETA: 45:00 - loss: 0.11 - ETA: 44:55 - loss: 0.11 - ETA: 44:51 - loss: 0.11 - ETA: 44:46 - loss: 0.11 - ETA: 44:41 - loss: 0.11 - ETA: 44:38 - loss: 0.11 - ETA: 44:34 - loss: 0.11 - ETA: 44:29 - loss: 0.11 - ETA: 44:26 - loss: 0.11 - ETA: 44:21 - loss: 0.11 - ETA: 44:17 - loss: 0.11 - ETA: 44:12 - loss: 0.11 - ETA: 44:07 - loss: 0.11 - ETA: 44:02 - loss

20000/20000 [==============================] - ETA: 23:09 - loss: 0.11 - ETA: 23:05 - loss: 0.11 - ETA: 23:00 - loss: 0.11 - ETA: 22:56 - loss: 0.11 - ETA: 22:51 - loss: 0.11 - ETA: 22:47 - loss: 0.11 - ETA: 22:42 - loss: 0.11 - ETA: 22:38 - loss: 0.11 - ETA: 22:33 - loss: 0.11 - ETA: 22:29 - loss: 0.11 - ETA: 22:24 - loss: 0.11 - ETA: 22:20 - loss: 0.11 - ETA: 22:15 - loss: 0.11 - ETA: 22:11 - loss: 0.11 - ETA: 22:07 - loss: 0.11 - ETA: 22:02 - loss: 0.11 - ETA: 21:58 - loss: 0.11 - ETA: 21:53 - loss: 0.11 - ETA: 21:49 - loss: 0.11 - ETA: 21:44 - loss: 0.11 - ETA: 21:40 - loss: 0.11 - ETA: 21:35 - loss: 0.11 - ETA: 21:31 - loss: 0.11 - ETA: 21:26 - loss: 0.11 - ETA: 21:22 - loss: 0.11 - ETA: 21:17 - loss: 0.11 - ETA: 21:12 - loss: 0.11 - ETA: 21:08 - loss: 0.11 - ETA: 21:03 - loss: 0.11 - ETA: 20:59 - loss: 0.11 - ETA: 20:54 - loss: 0.11 - ETA: 20:50 - loss: 0.11 - ETA: 20:45 - loss: 0.11 - ETA: 20:41 - loss: 0.11 - ETA: 20:36 - loss: 0.11 - ETA: 20:32 - loss: 0.11 - ETA: 20:28 - loss

In [31]:
score = model.evaluate(x_dev_, y_dev_, batch_size=32)
score

4106/4106 [==============================] - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 59 - ETA: 58 - ETA: 57 - ETA: 57 - ETA: 56 - ETA: 56 - ETA: 55 - ETA: 54 - ETA: 54 - ETA: 53 - ETA: 52 - ETA: 52 - ETA: 51 - ETA: 51 - ETA: 50 - ETA: 49 - ETA: 49 - ETA: 48 - ETA: 47 - ETA: 47 - ETA: 46 - ETA: 45 - ETA: 45 - ETA: 44 - ETA: 43 - ETA: 43 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 30 - ETA: 30 - ETA: 29 - ETA: 28 - ETA: 28 - ETA: 27 - ETA: 26 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 21 - ETA:

0.10926828805232013

In [32]:
y_test = model.predict(x_test, batch_size=32)

In [33]:
y_test[5040]

array([6.81032896e-01, 1.08781785e-01, 1.34173274e-01, 9.12191868e-02,
       1.59900308e-01, 1.90216929e-01, 8.61778557e-02, 8.34912956e-02,
       9.35764015e-02, 7.08179474e-02, 5.91582358e-02, 3.36692929e-02,
       3.42691839e-02, 4.85866964e-02, 5.04789948e-02, 3.41756344e-02,
       2.79759765e-02, 2.50927210e-02, 1.12388134e-02, 1.26051903e-02,
       7.28452206e-03, 5.99274039e-03, 2.83905864e-03, 2.92447209e-03,
       1.52266026e-03, 1.39629841e-03, 9.66310501e-04, 1.31630898e-03,
       1.09699368e-03, 1.05798244e-03, 1.05905533e-03, 6.06954098e-04,
       1.08975172e-03, 4.15176153e-04, 7.19785690e-04, 6.87181950e-04,
       3.24994326e-04, 6.37531281e-04, 8.35150480e-04, 7.83503056e-04,
       4.79727983e-04, 4.69923019e-04, 5.67972660e-04, 5.73337078e-04,
       4.56988811e-04, 6.48200512e-04, 3.98576260e-04, 2.94715166e-04,
       3.83317471e-04, 7.06553459e-04, 4.05997038e-04, 3.71575356e-04,
       4.18305397e-04, 4.87953424e-04, 4.88489866e-04], dtype=float32)